#### **Setup for fine-tuning and deploying Meta/llama2 model for Salesforce customer service dataset**

In this notebook, I use a qunatized version of llama2-base and PEFT/LoRA to reduce update size. The model is then fine-tuned on a Lightning.io T4 GPU (16GB) RAM where it trained well.

In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install torch==2.0.1 
!pip install transformers==4.32.1
!pip install datasets==2.14.4
!pip install peft==0.5.0
!pip install bitsandbytes==0.41.1
!pip install trl==0.7.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 18.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 11.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 766.6 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 8.3 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 6.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━

In [3]:
!pip install -U datasets

In [14]:
import json
import re
from pprint import pprint


import pandas as pd
import torch
from datasets import Dataset, load_dataset

from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel

from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig,
    TrainingArguments)

from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "meta-llama/Llama-2-7b-hf"


In [72]:
import os
os.environ["CURL_CA_BUNDLE"] = ""
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for Salesforce/dialogstudio contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Salesforce/dialogstudio
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [73]:
import json
def generate_text(data_point):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)
 
    conversation_text = create_conversation_text(data_point)

    try:
        example = {
            "conversation": conversation_text,
            "summary": summary,
            "text": generate_training_prompt(conversation_text, summary),
        }
        return example

    except Exception as e:
        return {
            "conversation": "",
            "summary": "",
            "text": ""
        }

DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()
 
 
def generate_training_prompt(
    conversation: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}
 
### Input:
{conversation.strip()}
 
### Response:
{summary}
""".strip()

In [74]:
def create_conversation_text(data_point):
    text = ""
    for item in data_point["log"]:
        user = clean_text(item["user utterance"])
        text += f"user: {user.strip()}\n"
 
        agent = clean_text(item["system response"])
        text += f"agent: {agent.strip()}\n"
 
    return text
 
 
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    return re.sub(r"\^[^ ]+", "", text)

def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_text)
        .remove_columns(
            [
                "original dialog id",
                "new dialog id",
                "dialog index",
                "original dialog info",
                "log",
                "prompt",
            ]
        )
    )

In [75]:
dataset["train"] = process_dataset(dataset["train"])
dataset["validation"] = process_dataset(dataset["validation"])

In [63]:
dataset["train"]

Dataset({
    features: ['conversation', 'summary', 'text'],
    num_rows: 879
})

In [28]:
notebook_login()

In [35]:
MODEL_NAME = "NousResearch/Llama-2-7b-chat-hf"

In [36]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
 
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        #new_model,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )
 
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
 
    return model, tokenizer

In [37]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [38]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [70]:
for example in dataset['validation']:
    if 'text' not in example.keys():
        print(example)

{'original dialog id': 'caae83a2ed59e4959d814ea567980226', 'new dialog id': 'TweetSumm--val--1', 'dialog index': 1, 'original dialog info': '{"summaries": {"extractive_summaries": [[{"is_agent": false, "sentences": ["@SpotifyCares hey, any explanation why the \\"Create similar playlist\\" function doesn\'t work anymore for me?"]}, {"is_agent": false, "sentences": ["@SpotifyCares i tried and it\'s still the same... moreover, my song history is always empty, so I can\'t find songs from previous Discover playlists :("]}, {"is_agent": true, "sentences": ["@160485 Could you DM us your account\'s email address or username?"]}, {"is_agent": true, "sentences": ["We\'ll take a look backstage /MT https://t.co/ldFdZRiNAt"]}], [{"is_agent": false, "sentences": ["@SpotifyCares hey, any explanation why the \\"Create similar playlist\\" function doesn\'t work anymore for me?"]}, {"is_agent": true, "sentences": ["@160485 Hi there, the cavalry\'s here!"]}, {"is_agent": true, "sentences": ["Does logging

In [64]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]
 
 
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [41]:
OUTPUT_DIR = "experiments"
 
%load_ext tensorboard
%tensorboard --logdir experiments/runs

/usr/bin/bash: load_ext: command not found


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.15.1 at http://localhost:6006/ (Press CTRL+C to quit)


In [76]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [77]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
22,2.016100,2.034687
44,1.895200,1.936691
66,1.713900,1.910567
88,1.816300,1.899833
110,1.695200,1.897427


TrainOutput(global_step=110, training_loss=1.9881060524420304, metrics={'train_runtime': 2723.5677, 'train_samples_per_second': 0.645, 'train_steps_per_second': 0.04, 'total_flos': 1.2937277221871616e+16, 'train_loss': 1.9881060524420304, 'epoch': 2.0})